In [1]:
# =============================================================================
# CELL 1: Configuration and Imports
# =============================================================================
import pandas as pd
import numpy as np
import os
import gc
import time
import warnings
from datetime import datetime

# Suppress fragmentation warning (cosmetic - doesn't affect correctness)
warnings.filterwarnings('ignore', message='DataFrame is highly fragmented')

# Input: merged pool files from full_universe_merge
INPUT_DIR = r"D:\ECB_ESMA_MERGED"

# Output: NEW folder for all country files
OUTPUT_DIR = r"D:\ECB_ESMA_BY_COUNTRY_ALL"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Chunk size for memory-efficient processing
CHUNK_SIZE = 100000

print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Chunk size: {CHUNK_SIZE:,} rows")
print()
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Input directory: D:\ECB_ESMA_MERGED
Output directory: D:\ECB_ESMA_BY_COUNTRY_ALL
Chunk size: 100,000 rows

Started at: 2025-12-12 22:27:19


In [2]:
# =============================================================================
# CELL 2: Country Detection Function (VERIFIED - DO NOT MODIFY)
# =============================================================================
def detect_country(df, pool_id):
    """
    Detect country from file data or pool ID.
    Priority order:
    1. RREL81 (Lender Country - ESMA)
    2. RREL84 (Originator Country - ESMA)
    3. RREC6 (Geographic Region NUTS - from AR128)
    4. RREL11 (Geographic Region Obligor NUTS)
    5. AR129 (ECB geographic - sometimes has NUTS codes)
    6. NPEL20/NPEL23 (NPE template country fields)
    7. Pool ID fallback (RMBM/RMBS + 2-letter country)
    """
    # Method 1: RREL81 (clean 2-letter code)
    if 'RREL81' in df.columns:
        vals = df['RREL81'].dropna().astype(str).unique()
        valid = [v for v in vals if len(v) == 2 and v.isalpha()]
        if valid:
            return valid[0].upper()
    
    # Method 2: RREL84 (clean 2-letter code)
    if 'RREL84' in df.columns:
        vals = df['RREL84'].dropna().astype(str).unique()
        valid = [v for v in vals if len(v) == 2 and v.isalpha()]
        if valid:
            return valid[0].upper()
    
    # Method 3: RREC6 (NUTS code - first 2 chars)
    if 'RREC6' in df.columns:
        vals = df['RREC6'].dropna().astype(str).unique()
        for v in vals:
            if len(v) >= 2 and v[:2].isalpha():
                return v[:2].upper()
    
    # Method 4: RREL11 (NUTS code - first 2 chars)
    if 'RREL11' in df.columns:
        vals = df['RREL11'].dropna().astype(str).unique()
        for v in vals:
            if len(v) >= 2 and v[:2].isalpha() and not v.startswith('ND'):
                return v[:2].upper()
    
    # Method 5: AR129 (ECB geographic - check for NUTS pattern)
    if 'AR129' in df.columns:
        vals = df['AR129'].dropna().astype(str).unique()
        for v in vals:
            if len(v) >= 2 and v[:2].isalpha() and not v.startswith('ND'):
                return v[:2].upper()
    
    # Method 6: NPEL20/NPEL23 (NPE template)
    for col in ['NPEL20', 'NPEL23']:
        if col in df.columns:
            vals = df[col].dropna().astype(str).unique()
            valid = [v for v in vals if len(v) == 2 and v.isalpha()]
            if valid:
                return valid[0].upper()
    
    # Method 7: FALLBACK - Extract from pool ID
    if pool_id.startswith('RMBM') or pool_id.startswith('RMBS'):
        country_from_id = pool_id[4:6]
        if country_from_id.isalpha():
            return country_from_id.upper()
    
    return 'UNKNOWN'

print("✓ Country detection function defined")

✓ Country detection function defined


In [3]:
# =============================================================================
# CELL 3: Scan All Files and Build Country Index
# =============================================================================
print("Scanning all merged files to build country index...")
print()

file_index = []  # List of {filepath, pool_id, folder, country, size_mb}

for folder in ['matched', 'ecb_only', 'esma_only']:
    folder_path = os.path.join(INPUT_DIR, folder)
    if not os.path.exists(folder_path):
        print(f"Warning: {folder_path} does not exist, skipping...")
        continue
    
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    print(f"Scanning {folder}: {len(files)} files...")
    
    for fname in files:
        filepath = os.path.join(folder_path, fname)
        file_size = os.path.getsize(filepath)
        
        # Read sample to detect country
        df_sample = pd.read_csv(filepath, nrows=100)
        pool_id = fname[:-4]  # Remove .csv extension
        country = detect_country(df_sample, pool_id)
        
        file_index.append({
            'filepath': filepath,
            'filename': fname,
            'pool_id': pool_id,
            'folder': folder,
            'country': country,
            'size_mb': file_size / (1024 * 1024)
        })

print(f"\nTotal files indexed: {len(file_index)}")

# Group by country and build statistics
country_stats = {}
for f in file_index:
    c = f['country']
    if c not in country_stats:
        country_stats[c] = {'count': 0, 'size_mb': 0, 'matched': 0, 'ecb_only': 0, 'esma_only': 0}
    country_stats[c]['count'] += 1
    country_stats[c]['size_mb'] += f['size_mb']
    country_stats[c][f['folder']] += 1

print("\nCountry distribution:")
print("-" * 90)
for country in sorted(country_stats.keys()):
    s = country_stats[country]
    print(f"{country}: {s['count']} files ({s['size_mb']/1024:.1f} GB) | matched={s['matched']}, ecb={s['ecb_only']}, esma={s['esma_only']}")

print()
print(f"Total countries to process: {len(country_stats)}")

Scanning all merged files to build country index...

Scanning matched: 22 files...
Scanning ecb_only: 36 files...
Scanning esma_only: 246 files...
Scanning ecb_only: 36 files...
Scanning esma_only: 246 files...

Total files indexed: 304

Country distribution:
------------------------------------------------------------------------------------------
BE: 30 files (52.4 GB) | matched=9, ecb=12, esma=9
DE: 15 files (50.2 GB) | matched=2, ecb=6, esma=7
ES: 87 files (11.0 GB) | matched=2, ecb=6, esma=79
FR: 30 files (90.9 GB) | matched=1, ecb=0, esma=29
IE: 24 files (4.0 GB) | matched=0, ecb=0, esma=24
IT: 22 files (8.9 GB) | matched=1, ecb=1, esma=20
NL: 72 files (43.2 GB) | matched=3, ecb=7, esma=62
PT: 15 files (0.7 GB) | matched=0, ecb=0, esma=15
UK: 8 files (54.4 GB) | matched=4, ecb=4, esma=0
UNKNOWN: 1 files (0.1 GB) | matched=0, ecb=0, esma=1

Total countries to process: 10

Total files indexed: 304

Country distribution:
--------------------------------------------------------------

In [4]:
# =============================================================================
# CELL 4: Define Country Merge Function (VERIFIED - RESUME SAFE)
# =============================================================================
def merge_country_files(country_code, country_files, output_dir, chunk_size=100000):
    """
    Merge all files for a single country into one CSV.
    Uses chunked streaming to handle large files efficiently.
    
    RESUME SAFE:
    - Skips if final output file already exists
    - Cleans up incomplete .tmp files from interrupted runs
    - Returns None if already completed (for skip tracking)
    
    Args:
        country_code: 2-letter country code (e.g., 'IT', 'DE')
        country_files: List of file info dicts for this country
        output_dir: Directory to write output file
        chunk_size: Number of rows per chunk (default 100000)
    
    Returns:
        dict with merge statistics, or None if skipped (already complete)
    """
    output_path = os.path.join(output_dir, f"{country_code}.csv")
    temp_path = output_path + ".tmp"
    
    # =========================================================================
    # RESUME SAFETY: Check if already completed
    # =========================================================================
    if os.path.exists(output_path):
        file_size_gb = os.path.getsize(output_path) / (1024**3)
        print(f"\n  ⏭️  SKIPPING {country_code}: Already exists ({file_size_gb:.2f} GB)")
        return None  # Signal that this was skipped
    
    # Clean up any incomplete temp files from previous interrupted runs
    if os.path.exists(temp_path):
        print(f"\n  🧹 Cleaning up incomplete temp file for {country_code}...")
        os.remove(temp_path)
    
    print(f"\n{'='*70}")
    print(f"PROCESSING: {country_code}")
    print(f"{'='*70}")
    print(f"Files to merge: {len(country_files)}")
    print(f"Output: {output_path}")
    
    # =========================================================================
    # PHASE 1: Scan all files to determine unified column schema
    # =========================================================================
    print("\nPhase 1: Building unified column schema...")
    all_columns = set()
    
    for f in country_files:
        df_header = pd.read_csv(f['filepath'], nrows=0)
        all_columns.update(df_header.columns.tolist())
    
    # Sort for consistent column ordering
    all_columns_sorted = sorted(list(all_columns))
    print(f"  Unified schema: {len(all_columns_sorted)} columns")
    
    # =========================================================================
    # PHASE 2: Stream-merge all files (chunked I/O)
    # =========================================================================
    print("\nPhase 2: Merging files (chunked streaming)...")
    start_time = time.time()
    total_rows = 0
    first_chunk = True
    
    for file_idx, f in enumerate(country_files):
        file_start = time.time()
        file_rows = 0
        
        print(f"  [{file_idx+1}/{len(country_files)}] [{f['folder']}] {f['filename'][:50]}... ({f['size_mb']:.0f} MB)")
        
        # Process file in chunks - NEVER load full file into memory
        for chunk in pd.read_csv(f['filepath'], chunksize=chunk_size, low_memory=False):
            # Add missing columns with NaN (fast operation)
            for col in all_columns_sorted:
                if col not in chunk.columns:
                    chunk[col] = np.nan
            
            # Reorder to unified schema
            chunk = chunk[all_columns_sorted]
            
            # Append to output (write header only on first chunk)
            chunk.to_csv(temp_path, mode='a', index=False, header=first_chunk)
            first_chunk = False
            
            file_rows += len(chunk)
            
            # Free chunk memory
            del chunk
        
        total_rows += file_rows
        file_elapsed = time.time() - file_start
        print(f"       -> {file_rows:,} rows in {file_elapsed:.1f}s")
        
        # Force garbage collection after each file
        gc.collect()
    
    # Rename temp to final
    os.replace(temp_path, output_path)
    
    total_elapsed = time.time() - start_time
    final_size_gb = os.path.getsize(output_path) / (1024**3)
    
    print(f"\n  ✓ COMPLETE: {country_code}")
    print(f"    Total rows: {total_rows:,}")
    print(f"    Columns: {len(all_columns_sorted)}")
    print(f"    File size: {final_size_gb:.2f} GB")
    print(f"    Time: {total_elapsed:.1f}s")
    
    return {
        'country': country_code,
        'files_merged': len(country_files),
        'total_rows': total_rows,
        'columns': len(all_columns_sorted),
        'size_gb': final_size_gb,
        'time_seconds': total_elapsed,
        'output_path': output_path
    }

print("✓ Country merge function defined")

✓ Country merge function defined


In [5]:
# =============================================================================
# CELL 5: Process ALL Countries (RESUME SAFE)
# =============================================================================
print("="*90)
print("PROCESSING ALL COUNTRIES (RESUME SAFE)")
print("="*90)
print()

# Get all countries to process
all_countries = sorted(country_stats.keys())
print(f"Countries to process: {all_countries}")
print(f"Total: {len(all_countries)} countries")
print()

# Check for already completed countries
already_done = []
for cc in all_countries:
    check_path = os.path.join(OUTPUT_DIR, f"{cc}.csv")
    if os.path.exists(check_path):
        already_done.append(cc)

if already_done:
    print(f"✓ Already completed (will skip): {already_done}")
    print(f"  Remaining to process: {len(all_countries) - len(already_done)} countries")
print()

# Track results
merge_results = []
skipped_countries = []
failed_countries = []

overall_start = time.time()

for country_idx, country_code in enumerate(all_countries):
    print(f"\n[{country_idx+1}/{len(all_countries)}] Starting {country_code}...")
    
    try:
        # Get files for this country
        country_files = [f for f in file_index if f['country'] == country_code]
        
        # Merge files (returns None if skipped due to existing file)
        result = merge_country_files(
            country_code=country_code,
            country_files=country_files,
            output_dir=OUTPUT_DIR,
            chunk_size=CHUNK_SIZE
        )
        
        if result is None:
            skipped_countries.append(country_code)
        else:
            merge_results.append(result)
        
    except KeyboardInterrupt:
        print(f"\n\n⚠️ INTERRUPTED during {country_code}!")
        print(f"   Cleaning up incomplete temp file...")
        temp_path = os.path.join(OUTPUT_DIR, f"{country_code}.csv.tmp")
        if os.path.exists(temp_path):
            os.remove(temp_path)
            print(f"   ✓ Removed {temp_path}")
        print(f"\n   Resume safe: Re-run this cell to continue from {country_code}")
        raise  # Re-raise to stop execution
        
    except Exception as e:
        print(f"\n  ✗ ERROR processing {country_code}: {str(e)}")
        failed_countries.append({'country': country_code, 'error': str(e)})
        # Clean up temp file on error too
        temp_path = os.path.join(OUTPUT_DIR, f"{country_code}.csv.tmp")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        continue

overall_elapsed = time.time() - overall_start

print("\n" + "="*90)
print("ALL COUNTRIES PROCESSED")
print("="*90)
print(f"\n  New: {len(merge_results)}, Skipped: {len(skipped_countries)}, Failed: {len(failed_countries)}")

PROCESSING ALL COUNTRIES (RESUME SAFE)

Countries to process: ['BE', 'DE', 'ES', 'FR', 'IE', 'IT', 'NL', 'PT', 'UK', 'UNKNOWN']
Total: 10 countries

✓ Already completed (will skip): ['BE']
  Remaining to process: 9 countries


[1/10] Starting BE...

  ⏭️  SKIPPING BE: Already exists (53.66 GB)

[2/10] Starting DE...

  🧹 Cleaning up incomplete temp file for DE...

PROCESSING: DE
Files to merge: 15
Output: D:\ECB_ESMA_BY_COUNTRY_ALL\DE.csv

Phase 1: Building unified column schema...
  Unified schema: 216 columns

Phase 2: Merging files (chunked streaming)...
  [1/15] [matched] RMBSDE000097100120083.csv... (2402 MB)

PROCESSING: DE
Files to merge: 15
Output: D:\ECB_ESMA_BY_COUNTRY_ALL\DE.csv

Phase 1: Building unified column schema...
  Unified schema: 216 columns

Phase 2: Merging files (chunked streaming)...
  [1/15] [matched] RMBSDE000097100120083.csv... (2402 MB)
       -> 2,159,244 rows in 99.8s
  [2/15] [matched] RMBSDE000556100120088.csv... (1145 MB)
       -> 2,159,244 rows in 99

In [6]:
# =============================================================================
# CELL 6: Summary Report
# =============================================================================
print("="*90)
print("FINAL SUMMARY REPORT")
print("="*90)
print()

print(f"Total processing time: {overall_elapsed/60:.1f} minutes")
print(f"Output directory: {OUTPUT_DIR}")
print()

# Success summary
print(f"✓ Newly processed: {len(merge_results)} countries")
print(f"⏭️  Skipped (already done): {len(skipped_countries)} countries")
if failed_countries:
    print(f"✗ Failed: {len(failed_countries)} countries")
print()

# Detailed results table
print("-"*90)
print(f"{'Country':<10} {'Files':<8} {'Rows':<15} {'Columns':<10} {'Size (GB)':<12} {'Time (s)':<10}")
print("-"*90)

total_rows_all = 0
total_size_all = 0

for r in merge_results:
    print(f"{r['country']:<10} {r['files_merged']:<8} {r['total_rows']:>12,} {r['columns']:<10} {r['size_gb']:<12.2f} {r['time_seconds']:<10.1f}")
    total_rows_all += r['total_rows']
    total_size_all += r['size_gb']

print("-"*90)
print(f"{'TOTAL':<10} {len(file_index):<8} {total_rows_all:>12,} {'':<10} {total_size_all:<12.2f} {overall_elapsed:<10.1f}")
print()

# Failed countries details
if failed_countries:
    print("\nFAILED COUNTRIES:")
    for f in failed_countries:
        print(f"  {f['country']}: {f['error']}")
    print()

# List output files
print("\nOUTPUT FILES:")
for r in merge_results:
    print(f"  {r['output_path']}")

print()
print(f"Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

FINAL SUMMARY REPORT

Total processing time: 214.8 minutes
Output directory: D:\ECB_ESMA_BY_COUNTRY_ALL

✓ Newly processed: 9 countries
⏭️  Skipped (already done): 1 countries

------------------------------------------------------------------------------------------
Country    Files    Rows            Columns    Size (GB)    Time (s)  
------------------------------------------------------------------------------------------
DE         15         53,242,506 216        53.73        2293.0    
ES         87         13,545,314 216        12.19        580.6     
FR         30         92,034,390 215        91.68        4443.1    
IE         24          4,849,990 196        4.31         204.0     
IT         22         11,029,781 216        9.24         489.8     
NL         72         39,952,401 217        45.37        1784.3    
PT         15            952,510 328        0.85         57.0      
UK         8          67,514,154 213        56.84        3023.2    
UNKNOWN    1             2

In [7]:
# =============================================================================
# CELL 7: Save Merge Log to JSON
# =============================================================================
import json

log_data = {
    'run_timestamp': datetime.now().isoformat(),
    'input_dir': INPUT_DIR,
    'output_dir': OUTPUT_DIR,
    'total_files_processed': len(file_index),
    'countries_processed': len(merge_results),
    'countries_failed': len(failed_countries),
    'total_rows': total_rows_all,
    'total_size_gb': total_size_all,
    'total_time_seconds': overall_elapsed,
    'results': merge_results,
    'failures': failed_countries,
    'country_stats': {k: dict(v) for k, v in country_stats.items()}
}

log_path = os.path.join(OUTPUT_DIR, 'merge_log.json')
with open(log_path, 'w') as f:
    json.dump(log_data, f, indent=2, default=str)

print(f"Merge log saved to: {log_path}")

Merge log saved to: D:\ECB_ESMA_BY_COUNTRY_ALL\merge_log.json


In [8]:
# =============================================================================
# CELL 8: Quick Verification of Output Files
# =============================================================================
print("="*90)
print("QUICK VERIFICATION OF OUTPUT FILES")
print("="*90)
print()

# Check each output file
for r in merge_results:
    filepath = r['output_path']
    country = r['country']
    
    if not os.path.exists(filepath):
        print(f"✗ {country}: File not found!")
        continue
    
    # Read sample and check
    df_sample = pd.read_csv(filepath, nrows=1000)
    file_size = os.path.getsize(filepath) / (1024**3)
    
    # Check source distribution
    source_dist = df_sample['source'].value_counts().to_dict() if 'source' in df_sample.columns else {}
    
    # Check column count
    col_count = len(df_sample.columns)
    
    print(f"✓ {country}: {file_size:.2f} GB, {col_count} cols, sources: {source_dist}")

print()
print("Verification complete!")

QUICK VERIFICATION OF OUTPUT FILES

✓ DE: 53.73 GB, 216 cols, sources: {'ECB': 1000}
✓ ES: 12.19 GB, 216 cols, sources: {'ECB': 1000}
✓ FR: 91.68 GB, 215 cols, sources: {'ECB': 1000}
✓ IE: 4.31 GB, 196 cols, sources: {'ESMA': 1000}
✓ IT: 9.24 GB, 216 cols, sources: {'ECB': 1000}
✓ NL: 45.37 GB, 217 cols, sources: {'ECB': 1000}
✓ NL: 45.37 GB, 217 cols, sources: {'ECB': 1000}
✓ PT: 0.85 GB, 328 cols, sources: {'ESMA': 1000}
✓ PT: 0.85 GB, 328 cols, sources: {'ESMA': 1000}
✓ UK: 56.84 GB, 213 cols, sources: {'ECB': 1000}
✓ UNKNOWN: 0.15 GB, 192 cols, sources: {'ESMA': 1000}

Verification complete!
✓ UK: 56.84 GB, 213 cols, sources: {'ECB': 1000}
✓ UNKNOWN: 0.15 GB, 192 cols, sources: {'ESMA': 1000}

Verification complete!
